In [1]:
import perceval as pcvl
import sympy as sp
import numpy as np

In [2]:
import perceval as pcvl
from perceval.components import PS, BS
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
plt.rcdefaults()

In [15]:
#Graph 1
# H1 = [[-2., 2.,  2.,  0.,  0.],
#       [0., -2.,  0.,  2.,  0.],
#       [0.,  0., -3.,  2.,  2.],
#       [0.,  0.,  0., -3.,  2.],
#       [0.,  0.,  0.,  0., -2.,]]
#Graph 2
# H1 = [[-2.0, 2.0, 0.0 ,2.0],
#       [0.0, -2.0, 2.0, 0.0],
#       [0.0, 0.0, -2.0, 2.0],
#       [0.0, 0.0, 0.0, -1.0]
#      ]

#Graph 3
H1 = [[-3.0, 2.0, 2.0, 2.0],
      [0.0, -2.0, 2.0, 0.0],
      [0.0, 0.0, -3.0, 2.0],
      [0.0, 0.0, 0.0 ,-2.0]
     ]

In [16]:
def parify_samples(samples, j):
    """apply the parity function to the samples"""
    def _parity(output, j):
        m = len(output)
        parity = [0]*m
        for i in range(0,m):
            parity[i] = (output[i] + j) % 2
        return pcvl.BasicState(parity)

    new_samples = pcvl.BSCount()
    for idx,sample in enumerate(samples):
        new_sample = _parity(sample,j)
        if new_sample in new_samples:
            new_samples[_parity(sample,j)] += samples[sample]
        else:
            new_samples[_parity(sample,j)] = samples[sample]
    return new_samples

def set_parameters_circuit(parameters_circuit, values):
    """set values of circuit parameters"""
    for idx, p in enumerate(parameters_circuit):
        parameters_circuit[idx].set_value(values[idx])

def compute_samples(circuit, input_state, nb_samples, j):
    """sample from the circuit"""
    p = pcvl.Processor("SLOS", circuit)
    p.with_input(input_state)
    p.min_detected_photons_filter(0)

    sampler = pcvl.algorithm.Sampler(p)
    samples = sampler.sample_count(nb_samples)['results']

    return parify_samples(samples,j)

In [17]:
def test_configuration(circuit, nb_modes, j, n, H, nb_samples):
    """output the samples for a given configuration of j and n (inludes minimisation of the loss function)"""
    parameters_circuit = circuit.get_parameters()

    input_state = pcvl.BasicState([1]*nb_modes)
    if n!=nb_modes:
        input_state = pcvl.BasicState([1]*(nb_modes-1)+[0])

    def loss(parameters):
        set_parameters_circuit(parameters_circuit, parameters)
        samples = compute_samples(circuit, input_state, nb_samples, j)
        E = 0
        for sample in samples:
            b = np.array([sample[i] for i in range(len(sample))])
            b_prime = np.dot(H, b)
            E += samples[sample]/nb_samples*np.dot(b.conjugate(), b_prime)

        return np.real(E)

    # init_parameters = [2*(np.pi)*random.random() for _ in parameters_circuit] # initialize with random initial parameters
    init_parameters = [np.pi for _ in parameters_circuit] # initialize with a good guess
    best_parameters = minimize(loss, init_parameters, method='Powell', bounds=[(0,2*np.pi) for _ in init_parameters]).x
    set_parameters_circuit(parameters_circuit, best_parameters)
    samples = compute_samples(circuit, input_state, nb_samples, j)
    return samples

def shortest_path(H, nb_samples):
    """run the universal circuit and optimize the parameters"""
    nb_modes = len(H)
    circuit = pcvl.Circuit.generic_interferometer(
        nb_modes,
        lambda i: BS(theta=pcvl.P(f"theta{i}"),phi_tr=pcvl.P(f"phi_tr{i}")),
        phase_shifter_fun_gen=lambda i: PS(phi=pcvl.P(f"phi{i}")))
    js = [0,1]
    ns = [nb_modes, nb_modes-1]
    configuration_samples = []
    for j in js:
        for n in ns:
            current_sample = test_configuration(circuit, nb_modes, j, n, H, nb_samples)
            print(f"Configuration for (j,n)=({j},{n})")
            print(current_sample)
            configuration_samples.append(([j,n],current_sample))

    return configuration_samples

In [18]:
nb_samples = 10000
data = shortest_path(H1, nb_samples)

Configuration for (j,n)=(0,4)
{
  |1,1,0,0>: 8
  |0,0,0,0>: 4
  |0,1,0,1>: 9981
  |1,1,1,1>: 1
  |0,1,1,0>: 1
  |0,0,1,1>: 4
}
Configuration for (j,n)=(0,3)
{
  |0,1,0,0>: 2
  |1,0,0,0>: 2987
  |1,1,1,0>: 1
  |1,1,0,1>: 3535
  |0,0,1,0>: 1602
  |0,1,1,1>: 1873
  |0,0,0,1>: 1
}
Configuration for (j,n)=(1,4)
{
  |0,0,1,1>: 2288
  |0,1,0,1>: 9
  |1,0,0,1>: 5642
  |1,0,1,0>: 2028
  |1,1,1,1>: 11
  |1,1,0,0>: 9
  |0,1,1,0>: 2
  |0,0,0,0>: 10
}
Configuration for (j,n)=(1,3)
{
  |0,1,1,1>: 6591
  |0,0,1,0>: 325
  |1,0,1,1>: 1
  |1,1,0,1>: 2934
  |1,0,0,0>: 147
}
